In [1]:
import warnings

import pandas as pd
from pymatgen.core import Composition, Lattice, Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from tqdm import tqdm

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")
    warnings.filterwarnings("ignore", category=category, module="pymatgen")

In [2]:
import tensorflow as tf

tf.config.experimental.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
data = pd.read_html("http://en.wikipedia.org/wiki/Lattice_constant")[0]
data = data[
    ~data["Crystal structure"].isin(
        ["Hexagonal", "Wurtzite", "Wurtzite (HCP)", "Orthorombic", "Tetragonal perovskite", "Orthorhombic perovskite"]
    )
]
data.rename(columns={"Lattice constant (Å)": "a"}, inplace=True)
data.drop(columns=["Ref."], inplace=True)
data["a"] = data["a"].map(float)
data = data[["Material", "Crystal structure", "a"]]
data = data[data["Material"] != "NC0.99"]

additional_fcc = """10 Ne 4.43 54 Xe 6.20
13 Al 4.05 58 Ce 5.16
18 Ar 5.26 70 Yb 5.49
20 Ca 5.58 77 Ir 3.84
28 Ni 3.52 78 Pt 3.92
29 Cu 3.61 79 Au 4.08
36 Kr 5.72 82 Pb 4.95
38 Sr 6.08 47 Ag 4.09
45 Rh 3.80 89 Ac 5.31
46 Pd 3.89 90 Th 5.08"""

additional_bcc = """3 Li 3.49 42 Mo 3.15
11 Na 4.23 55 Cs 6.05
19 K 5.23 56 Ba 5.02
23 V 3.02 63 Eu 4.61
24 Cr 2.88 73 Ta 3.31
26 Fe 2.87 74 W 3.16
37 Rb 5.59 41 Nb 3.30"""


def add_new(str_, structure_type, df):
    tokens = str_.split()
    new_crystals = []
    for i in range(int(len(tokens) / 3)):
        el = tokens[3 * i + 1].strip()
        if el not in df["Material"].values:
            new_crystals.append([tokens[3 * i + 1], structure_type, float(tokens[3 * i + 2])])
    df2 = pd.DataFrame(new_crystals, columns=data.columns)
    return pd.concat([df, df2])


data = add_new(additional_fcc, "FCC", data)
data = add_new(additional_bcc, "BCC", data)
data.set_index("Material", inplace=True)
print(data)

             Crystal structure         a
Material                                
C (diamond)      Diamond (FCC)  3.567000
Si               Diamond (FCC)  5.431021
Ge               Diamond (FCC)  5.658000
AlAs         Zinc blende (FCC)  5.660500
AlP          Zinc blende (FCC)  5.451000
...                        ...       ...
K                          BCC  5.230000
Ba                         BCC  5.020000
Eu                         BCC  4.610000
Cr                         BCC  2.880000
Rb                         BCC  5.590000

[92 rows x 2 columns]


In [4]:
predicted = []
mp = []
mpr = MPRester()
relaxer = Relaxer()  # This loads the default pre-trained model

# warnings.filterwarnings(action="ignore", category=UserWarning, module="tensorflow")

for formula, v in tqdm(data.iterrows(), total=len(data)):
    formula = formula.split()[0]
    c = Composition(formula)
    els = sorted(c.elements)
    cs = v["Crystal structure"]

    # We initialize all the crystals with an arbitrary lattice constant of 5 angstroms.
    if "Zinc blende" in cs:
        s = Structure.from_spacegroup("F-43m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.25, 0.25, 0.75]])
    elif "Halite" in cs:
        s = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0, 0]])
    elif "Caesium chloride" in cs:
        s = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif "Cubic perovskite" in cs:
        s = Structure(
            Lattice.cubic(5),
            [els[0], els[1], els[2], els[2], els[2]],
            [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]],
        )
    elif "Diamond" in cs:
        s = Structure.from_spacegroup("Fd-3m", Lattice.cubic(5), [els[0]], [[0.25, 0.75, 0.25]])
    elif "BCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 2, [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]])
    elif "FCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 4, [[0.0, 0.0, 0.0], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]])
    else:
        predicted.append(0)
        mp.append(0)
        continue

    relax_results = relaxer.relax(s)

    final_structure = relax_results["final_structure"]

    predicted.append(final_structure.lattice.a)

    try:
        entries = mpr.get_entries(
            {"pretty_formula": s.composition.reduced_formula},
            inc_structure=True,
            property_data=["e_above_hull"],
            sort_by_e_above_hull=True,
        )
        for e in entries:
            try:
                sga = SpacegroupAnalyzer(e.structure)
                sga2 = SpacegroupAnalyzer(final_structure)
                if sga.get_space_group_number() == sga2.get_space_group_number():
                    conv = sga.get_conventional_standard_structure()
                    mp.append(conv.lattice.a)
                    break
            except:
                pass
        else:
            raise RuntimeError
    except Exception as ex:
        mp.append(0)
        import traceback

        traceback.print_exc()

data["MP a"] = mp
data["Predicted a"] = predicted

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-07-10 12:21:55.783584: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-10 12:21:55.783870: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
  0%|          | 0/92 [00:00<?, ?it/s]2022-07-10 12:21:59.922474: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-10 12:21:59.932112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
  0%|          | 0/92 [00:04<?, ?it/s]


NotFoundError: Graph execution error:

Detected at node 'gradients/m3g_net/graph_network_layer/gated_atom_update/UnsortedSegmentSum_grad/and' defined at (most recent call last):
    File "/opt/homebrew/Caskroom/miniconda/base/envs/m3gnet/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Caskroom/miniconda/base/envs/m3gnet/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/homebrew/Caskroom/miniconda/base/envs/m3gnet/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/homebrew/Caskroom/miniconda/base/envs/m3gnet/lib/python3.10/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/opt/homebrew/Caskroom/miniconda/base/envs/m3gnet/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/homeb

In [21]:
data["% error vs Expt"] = (data["Predicted a"] - data["a"]) / data["a"]
data["% error vs MP"] = (data["Predicted a"] - data["MP a"]) / data["MP a"]

In [22]:
data.sort_index().style.format({"% error vs Expt": "{:,.2%}", "% error vs MP": "{:,.2%}"}).background_gradient()

/Users/janosh/.venv/py310/lib/python3.10/site-packages/pandas/io/formats/style.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  norm = mpl.colors.Normalize(smin - (rng * low), smax + (rng * high))


,Crystal structure,a,MP a,Predicted a,% error vs Expt,% error vs MP
Material,,,,,,
Ac,FCC,5.310000,5.662256,5.664600,6.68%,0.04%
Ag,FCC,4.079000,4.160548,4.167017,2.16%,0.16%
Al,FCC,4.046000,4.038930,4.041083,-0.12%,0.05%
AlAs,Zinc blende (FCC),5.660500,5.733756,5.730271,1.23%,-0.06%
AlP,Zinc blende (FCC),5.451000,5.507112,5.503462,0.96%,-0.07%
AlSb,Zinc blende (FCC),6.135500,6.233760,6.228168,1.51%,-0.09%
Ar,FCC,5.260000,5.640772,5.627447,6.99%,-0.24%
Au,FCC,4.065000,4.171288,4.174314,2.69%,0.07%
BN,Zinc blende (FCC),3.615000,3.626002,3.624853,0.27%,-0.03%
